# Reserve and configure FABRIC resources for reproducing "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

## Check if slice already exists

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)
     

#slice = fablib.new_slice(name=slice_name)

This cell will try to identify a site that has sufficient resources for your experiment. But, you should still check in the [FABRIC Portal](https://portal.fabric-testbed.net/resources/all) to make sure it is not in maintenance or out of service - if so, you should run the cell again until you get a site that is ready to use.

## Defining resource configuration

In [ ]:
sender_common = {'cores':4, 'ram':32, 'disk':500, 'image':'default_ubuntu_18'}
receiver_common = {'cores':4, 'ram':32, 'disk':100, 'image':'default_ubuntu_18'}

router_node_conf = [{'name': "router", 'cores':32, 'ram' : 256, 'disk':500, 'image':'default_ubuntu_22'}]
sender_node_conf = [{'name': f"sender-{i}", **sender_common} for i in range(n_endpoints)]
receiver_node_conf = [{'name': f"receiver-{i}", **receiver_common} for i in range(n_endpoints)]

node_conf = router_node_conf + sender_node_conf + receiver_node_conf

net_conf = [
    {
        "name" : "link-sender", 
        "subnet":"10.10.1.0/24",
        "nodes": [{"name": "sender-" + str(i), "addr": "10.10.1.1"+ str(i)} for i in range(n_endpoints)] + [{"name": "router", "addr" : "10.10.1.1"}],
        "idx":0
    },   
    {
        "name" : "link-receiver", 
        "subnet":"10.10.2.0/24",
        "nodes": [{"name": "receiver-" + str(i), "addr": "10.10.2.1" + str(i)} for i in range(n_endpoints)]+ [{"name": "router", "addr" : "10.10.2.1"}],
        
        "idx":1
    }
]
               
route_conf = [
    {
        "addr":"10.10.2.0/24", "gw": "10.10.1.1", 
        "nodes" : ["sender-"+ str(i) for i in range(n_endpoints)]
    },
    {
        "addr":"10.10.1.0/24", "gw": "10.10.2.1", 
        "nodes" : ["receiver-" + str(i) for i in range(n_endpoints)]
    }
]

exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }


## Reserve Resources

In [ ]:
exp_requires = {'core': 2*n_endpoints*4+32, 'nic': 2*n_endpoints*1, 'connectx5': 1}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) and
        (fablib.resources.get_component_available(site_name, 'SmartNIC-ConnectX-5') > 1.2*exp_requires['connectx5'])   ):
        break

fablib.show_site(site_name)

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

While it is being prepared, the “State” of the slice will appear as “Configuring”. When it is ready, the “State” of the slice will change to “StableOK”. You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [ ]:
slice.submit()


In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

## Extend the slice

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 4 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

## Configuring the slice

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

In [ ]:
# Also install `iperf3` on sender and receiver hosts:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

from ipaddress import ip_address, IPv6Address
for n in sender_nodes:
    if type(ip_address(n.get_management_ip())) is IPv6Address:
        n.upload_file('nat64.sh', 'nat64.sh')
        stdout, stderr = n.execute(f'chmod +x nat64.sh && ./nat64.sh')
    n.upload_file("iperf-parallel-senders.sh","iperf-parallel-senders.sh")
    n.execute("sudo apt-get update; sudo apt-get -y install build-essential ")
    n.execute_thread("sudo modprobe tcp_bbr")
    #n.execute("sudo rm -r /iperf")
    n.execute_thread("sudo git clone https://github.com/vinitaparasrampuria/iperf.git /iperf; sudo chmod +x /iperf/configure; sudo bash /iperf/configure;\
                     sudo make; sudo make check; sudo make install; sudo ldconfig")
              
for n in receiver_nodes:
    n.execute_thread("sudo apt update; sudo apt -y install iperf3;  sudo modprobe tcp_bbr")

In [ ]:
for n in sender_nodes:
    n.execute("sudo apt-get update; sudo apt-get install -y python3-pip ethtool netcat moreutils ",quiet=True)
    n.execute("sudo python3 -m pip install scikit-learn numpy pandas matplotlib seaborn", quiet=True)


In [ ]:
for n in sender_nodes:
    n.execute("iperf3 -version")

In [ ]:
for n in receiver_nodes:
    n.upload_file("iperf-parallel-servers.sh","iperf-parallel-servers.sh")

## Draw the topology

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
all_nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
print(ifaces[0])
print(ifaces[1])
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]
print(edges)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(all_nodes),len(all_nodes)))
G = nx.Graph()
G.add_nodes_from(all_nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in all_nodes], 
        node_size=[len(n[0])*400 for n in all_nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);
     

## Delete your slice

In [ ]:
slice.delete()

# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()